# Miniproyecto 4

### David Soto 17551 & Miguel Valle 17102

In [ ]:
### Miniproyecto 4
### Mancala Montecarlo
### David Soto 17551 & Miguel Valle 17102
import random
import copy

### Definición de Constantes y Tablero

In [ ]:
### Constantes
PROPORCION = 1.0 / 12.0

NOOB = 0
AVANZADO = 500
PRO = 10000

IDJUGADOR = True
IDAI = False

GANAJUGADOR = 'Gano Jugador Humano'
GANAAI = 'Gano la AI'
EMPATE = 'Empate'

POSICIONESOPUESTAS = {
    0:12,
    1:11,
    2:10,
    3:9,
    4:8,
    5:7,
    7:5,
    8:4,
    9:3,
    10:2,
    11:1,
    12:0
}

### Tablero
tablero = [4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0]

### Función de elección uniforme de tiros posibles, imprimir tablero y evaluar resultado 

In [ ]:
### Eleccion uniforme de los tiros posibles
def eleccionMontecarlo(disponibles):
    return random.choice(disponibles)

### Recibe un tablero para poder imprimirlo en pantalla
def printTablero(tablero):
    print('#-----------------------TABLERO ACTUAL-----------------------#')
    print('           HUMAN SIDE')
    print(' ',12,' ',11,' ',10,'  ',9,'  ',8,'  ',7,'  ')
    print('  ',tablero[12],'  ',tablero[11],'  ',tablero[10],'  ',tablero[9],'  ',tablero[8],'  ',tablero[7],'  ')
    print(tablero[13],'                            ',tablero[6])
    print('  ',tablero[0],'  ',tablero[1],'  ',tablero[2],'  ',tablero[3],'  ',tablero[4],'  ',tablero[5],'  ')
    print('  ',1,'  ',2,'  ',3,'  ',4,'  ',5,'  ',6,'  ')
    print('            AI SIDE')

### Recibe un tablero y determina si el juego ha finalizado o no
### Tambien devuelve en caso de finalizado el juego un String con el texto del ganador
def evaluacionResultado(tablero):
    if (tablero[0] == tablero[1] == tablero[2] == tablero[3] == tablero[4] == tablero[5] == 0) or (tablero[7] == tablero[8] == tablero[9] == tablero[10] == tablero[11] == tablero[12] == 0):
        punteoFinalAI = tablero[6] + tablero[5] + tablero[4] + tablero[3] + tablero[2] + tablero[1] + tablero[0]
        punteoFinalHumano = tablero[13] + tablero[12] + tablero[11] + tablero[10] + tablero[9] + tablero[8] + tablero[7]
        if punteoFinalAI > punteoFinalHumano:
            marcador = "Resultado Final:\nHumano " + str(punteoFinalHumano) + " - " + str(punteoFinalAI) + " AI"
            return True, GANAAI, marcador
        if punteoFinalHumano > punteoFinalAI:
            marcador = "Resultado Final:\nHumano " + str(punteoFinalHumano) + " - " + str(punteoFinalAI) + " AI"
            return True, GANAJUGADOR, marcador
        else:
            marcador = "Resultado Final:\nHumano " + str(punteoFinalHumano) + " - " + str(punteoFinalAI) + " AI"
            return True, EMPATE, marcador
    else:
        return False, EMPATE, ''

### Función para hacer jugadas

In [ ]:
### Recibe un tablero y el identificador del jugador, para realizar una jugada
### Devuelve el tablero actualizado, y el proximo turno
def jugada(movida, tablero, jugador, ai=False):
    ### Reglas del Mancala
    if (movida <= 6) and (ai == False):
        movida = movida - 1
    
    ### Movida de AI
    if not jugador:
        ### Mover las piedras
        piedras = tablero[movida]
        tablero[movida] = 0
        for i in range(piedras):
            tablero[(movida + i + 1) % 14] = tablero[(movida + i + 1) % 14] + 1
            
            if i == (piedras - 1):
                ### Revisar si se repite turno
                if ((movida + i + 1) % 14) == 6:
                    jugador = jugador
                ### Revisar si nos robamos la fichas
                elif (tablero[(movida + i + 1) % 14] == 1) and (((movida + i + 1) % 14) != 13) and (((movida + i + 1) % 14) < 6):
                    tablero[(movida + i + 1) % 14] = 0
                    puntos = tablero[POSICIONESOPUESTAS[(movida + i + 1) % 14]]
                    tablero[POSICIONESOPUESTAS[(movida + i + 1) % 14]] = 0
                    tablero[6] = 1 + puntos + tablero[6]
                    jugador = not jugador
                else:
                    jugador = not jugador
    ### Movida Humano
    else:
        ### Mover las piedras
        piedras = tablero[movida]
        tablero[movida] = 0
        for i in range(piedras):
            tablero[(movida + i + 1) % 14] = tablero[(movida + i + 1) % 14] + 1

            if i == (piedras - 1):
                ### Revisar si se repite turno
                if ((movida + i + 1) % 14) == 13:
                    jugador = jugador
                ### Revisar si nos robamos la fichas
                elif (tablero[(movida + i + 1) % 14] == 1) and (((movida + i + 1) % 14) != 6) and (((movida + i + 1) % 14 > 6)):
                    tablero[(movida + i + 1) % 14] = 0
                    puntos = tablero[POSICIONESOPUESTAS[(movida + i + 1) % 14]]
                    tablero[POSICIONESOPUESTAS[(movida + i + 1) % 14]] = 0
                    tablero[13] = 1 + puntos + tablero[13]
                    jugador = not jugador
                else:
                    jugador = not jugador      

    return tablero, jugador

### Función para algoritmo montecarlo

In [ ]:
### Recibe un tablero y la dificultad del juego, realiza jugadas aleatorias hasta terminar el juego
### Una vez terminado el juego hace un conteo y proporcion sobre en que jugada gana mas veces
### Devuelve una jugada definitiva en forma de una posicion del arreglo tablero
def algoritmoMontecarloMancalo(tablero, dificultad):
    opcionesAI = [0, 1, 2, 3, 4, 5]
    opcionesJugador = [7, 8, 9, 10, 11, 12]
    punteo = [0, 0, 0, 0, 0, 0]
    seleccionado = [0, 0, 0, 0, 0, 0]
    proporciones = [0, 0, 0, 0, 0, 0]
    opcionesAIFiltrado = []
    opcionesJugadorFiltrado = []
    opcionesAIFiltradoInterno = []
    opcionesJugadorFiltradoInterno = []

    ### Filtrando las casillas posibles
    for opcion in opcionesAI:
        if tablero[opcion] != 0:
            opcionesAIFiltrado.append(opcion)
            opcionesAIFiltradoInterno.append(opcion)

    for opcion in opcionesJugador:
        if tablero[opcion] != 0:
            opcionesJugadorFiltrado.append(opcion)
            opcionesJugadorFiltradoInterno.append(opcion)       

    ### Repeticion del juego
    for i in range(dificultad):
        tableroSimulacion = copy.deepcopy(tablero)
        juegoTerminado = False
        turno = False
        primerMovimiento = None
        if turno:
            primerMovimiento = eleccionMontecarlo(opcionesJugadorFiltrado)
        else:
            primerMovimiento = eleccionMontecarlo(opcionesAIFiltrado)
        mov = primerMovimiento
        while not juegoTerminado:
            ### Turnos simulados
            tableroSimulacion, turno = jugada(mov, tableroSimulacion, turno, True)
            juegoTerminado, resultado, marcador = evaluacionResultado(tableroSimulacion)

            ### Si el juego no termina
            if not juegoTerminado:
                opcionesAIFiltradoInterno = []
                opcionesJugadorFiltradoInterno = []

                ### Filtrando las casillas posibles
                for opcion in opcionesAI:
                    if tableroSimulacion[opcion] != 0:
                        opcionesAIFiltradoInterno.append(opcion)

                for opcion in opcionesJugador:
                    if tableroSimulacion[opcion] != 0:
                        opcionesJugadorFiltradoInterno.append(opcion)  

                if turno:
                    mov = eleccionMontecarlo(opcionesJugadorFiltradoInterno)
                else:
                    mov = eleccionMontecarlo(opcionesAIFiltradoInterno)

            #### Si el juego termina
            else:
                posicion = opcionesAIFiltrado.index(primerMovimiento)
                if resultado == GANAAI:
                    punteo[posicion] += 1
                    seleccionado[posicion] += 1
                else:
                    seleccionado[posicion] += 1

    mejorJugada = None

    if dificultad == NOOB:
        ### Jugada Noob
        mejorJugada = eleccionMontecarlo(opcionesAIFiltrado)
    else:
        ### Revisar la mejor jugada propocional
        for i in range(len(proporciones)):
            if seleccionado[i] != 0:
                proporciones[i] = float(punteo[i]) / float(seleccionado[i])

        mejorJugada = opcionesAIFiltrado[proporciones.index(max(proporciones))]

    return mejorJugada

### Main

In [ ]:
#-----------------Main del Juego Mancala-----------------#
print("Bienvenido a Mancala")
tablero = [4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0]
dificultad = -1
turno = 1
jugar = True

while jugar:
    ### Eleccion de dificultad
    while dificultad == -1:
        try:
            opcionDificultad = int(input("Elije la dificultad de tu oponente (Escribe el numero)\n1. Noob\n2. Avanzado\n3. Pro\n4. Salir\n>>"))
            if opcionDificultad == 1:
                dificultad = NOOB
                print("Dificultad Elegida > Noob")
            elif opcionDificultad == 2:
                dificultad = AVANZADO
                print("Dificultad Elegida > Avanzado")
            elif opcionDificultad == 3:
                dificultad = PRO
                print("Dificultad Elegida > Pro")
            elif opcionDificultad == 4:
                dificultad = -1
                break
            else:
                print("Opcion invalida!!!")
        except:
            print("Opcion invalida!!! -.-")

    ### Comienza el juego
    finJuego = False
    if dificultad != -1:
        print("Juguemos")
        print('#-----------------------TABLERO INICIAL-----------------------#')
        printTablero(tablero)
        ### Elegimos el turno al azar de quien comienza
        turno = random.choice([True, False])
        if turno:
            print("Comienza el Jugador Humano")
        else:
            print("Comienza AI")
        #turno = False
        ### Juego por Turnos
        while not finJuego:
            ### Turno del Jugador Humano
            if turno == True:
                movida = None
                while movida == None:
                    try:
                        movida = int(input("Elije el numero de casilla que deseas mover (Escribe el numero)>>"))
                        movi = movida
                        ### Determinar si la casilla es del lado humano y si tiene montoncitos
                        if movi <= 6:
                            movi = movi - 1

                        if (movida >= 7) and (movida <= 12) and (tablero[movi] != 0):
                            tablero, turno = jugada(movida, tablero, turno)
                        else:
                            print("Opcion invalida, prueba de nuevo!!!")
                            movida = None

                    except:
                        print("Opcion invalida, prueba de nuevo!!!")
                        movida = None

            ### Turno del Jugador AI
            else:
                movida = algoritmoMontecarloMancalo(tablero, dificultad)
                tablero, turno = jugada(movida, tablero, turno, True)

            printTablero(tablero)
            finJuego, resultado, marcador = evaluacionResultado(tablero)
            if finJuego:
                print("El juego ha terminado")
                print(marcador)
                print("El resultado es", resultado)

        try:
            opcionJugar = int(input("Desea jugar otra vez (Escribe el numero)\n1. Si\n2. No\n>>"))
            if opcionJugar == 1:
                jugar = True
                dificultad = -1
                tablero = [4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0]
                
            elif opcionJugar == 2:
                jugar = False
        except:
            jugar = False
    else:
        break

print("Hasta pronto")